These are scripts utilizing boto3 to view spot terminations and logs (which are currently way too long to inspect).

They are the work of [Andrew Johnston](https://github.com/asjohnston-asf).

In [4]:
import os
import boto3

In [5]:
os.environ['AWS_PROFILE'] = 'tibet'
batch = boto3.client('batch')

In [7]:
params = {
    'jobQueue': 'BatchJobQueue-8YxF2wT5tW3nPcry',
    'jobStatus': 'FAILED',
    'maxResults': 100,
}

terminations = []
exited = []

paginator = batch.get_paginator('list_jobs')
page_iterator = paginator.paginate(**params)
for k, page in enumerate(page_iterator):
    print(k)
    job_ids = [job['jobId'] for job in page['jobSummaryList']]
    response = batch.describe_jobs(jobs=job_ids)
    for job in response['jobs']:
        print(f'{job["jobName"]} {job["statusReason"]}')
    if k > 2:
        break

0
8601265e-4d50-46e0-9351-d605f3b05fbe Host EC2 (instance i-0be1a815b331961f1) terminated.
4bb115a1-542a-42e3-9688-e9e7a74952ef Host EC2 (instance i-0be1a815b331961f1) terminated.
ae8ac8d8-5908-4608-86eb-b20c9d166c78 Host EC2 (instance i-02ceab5d663276377) terminated.
4580d813-5601-40b2-9ee1-54af69584c34 Host EC2 (instance i-079f9ce806ebc4ff7) terminated.
a9795dac-8971-4f29-b4f5-b9946fb9d28d Host EC2 (instance i-0b28e262c3a2e3841) terminated.
526a10d0-91a8-4f44-90b9-a510c215d889 Host EC2 (instance i-05c8e54a972e3032a) terminated.
bf1a6857-058c-43f1-96a9-757458d55ba3 Host EC2 (instance i-08d83fd08f088877c) terminated.
893e20c0-2170-40f4-a2d1-3ffa9e74f867 Host EC2 (instance i-017d915a0c1dc7e95) terminated.
be832e42-973c-4b6a-85e9-6593533bca8d Host EC2 (instance i-0c2a0e7607cd17efd) terminated.
31b24773-1c62-4def-b391-af2339daf159 Host EC2 (instance i-044da19b07f4adc98) terminated.
20ce7359-6a44-44fc-948c-af23be531980 Host EC2 (instance i-03f1295c0931e8c44) terminated.
f6319e03-ea29-45a2-

In [8]:
params = {
    'jobQueue': 'BatchJobQueue-8YxF2wT5tW3nPcry',
    'jobStatus': 'FAILED',
    'maxResults': 100,
}

terminations = []
exited = []

paginator = batch.get_paginator('list_jobs')
page_iterator = paginator.paginate(**params)
for k, page in enumerate(page_iterator):
    print(k)
    job_ids = [job['jobId'] for job in page['jobSummaryList']]
    response = batch.describe_jobs(jobs=job_ids)
    terminations += [job['jobName'] for job in response['jobs'] if 'terminated.' in job["statusReason"]]
    exited += [job['jobName'] for job in response['jobs'] if 'task exited' in job["statusReason"]]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [9]:
float(len(terminations)) / (len(exited) + len(terminations))

0.9063144861741642

In [10]:
(len(exited) + len(terminations))

2423

In [11]:

from os import environ
import json

import boto3
import requests

session = requests.Session()


def get_log_message(job_id):
    log_url = f'https://{domain}/{job_id}/{job_id}.log'
    # THIS IS THE BRILLIANT PART!
    # Request to only read the last line 
    headers = {'Range': 'bytes=-1000'}
    response = session.get(log_url, headers=headers)
    try:
        response.raise_for_status()
        message = response.text.split('\n')[-1]
        return message
    except Exception:
        return ''

environ['AWS_PROFILE'] = 'tibet'
state_machine_arn = 'arn:aws:states:us-west-2:015983320819:stateMachine:StepFunction-uZ45938oodIw'
domain = 'hyp3-tibet-jpl-contentbucket-81rn23hp7ppf.s3.us-west-2.amazonaws.com'

sfn = boto3.client('stepfunctions')
max_results = 40


response = sfn.list_executions(stateMachineArn=state_machine_arn, statusFilter='FAILED', maxResults=max_results)
executions = response['executions']

for execution in executions:
    response = sfn.describe_execution(executionArn=execution['executionArn'])
    payload = json.loads(response['input'])
    message = get_log_message(payload['job_id'])
    print(f'{payload["job_id"]}  {response["stopDate"].strftime("%Y-%m-%d %H:%M:%S%z")}  '
          f'{payload["job_type"]:11}  {payload["user_id"]:25}  {message}')

d975293c-5cfd-4390-8464-270aba607509  2024-03-01 15:44:15-0800  INSAR_ISCE   access_cloud_based_insar   ValueError: Reference and/or secondary dates were not connected in their coverage (multipolygons)
44115abe-c62e-4ae3-ac66-8743821ac9b6  2024-03-01 15:43:13-0800  INSAR_ISCE   access_cloud_based_insar   ValueError: Reference and/or secondary dates were not connected in their coverage (multipolygons)
15da1a4e-7250-4e1b-91f0-0a22b62ddbec  2024-02-29 18:08:57-0800  INSAR_ISCE   access_cloud_based_insar   snaphu v1.4.2
8bad3178-0e16-42a9-b280-e0a163ab3532  2024-02-29 17:18:11-0800  INSAR_ISCE   access_cloud_based_insar   ValueError: TopsApp failed at step: fineresamp
e73373fb-a1c1-416e-a054-2d8411feb7a9  2024-02-29 16:28:00-0800  INSAR_ISCE   access_cloud_based_insar   Initializing flows with MCF algorithm
f9eeea56-8e7c-4770-ab12-16d62a8ce08d  2024-02-29 16:28:00-0800  INSAR_ISCE   access_cloud_based_insar   Initializing flows with MCF algorithm
6c2866a1-fea6-4106-8ab0-f7fc4ba9cc36  2024-